# Exemple niveau 1

In [8]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

def entete(canvas, doc):
    canvas.saveState()
    canvas.setFont('Helvetica-Bold', 10)
    canvas.drawString(2 * cm, 28 * cm, "Copropriété Monica – Historique RGD")
    canvas.setFont('Helvetica', 8)
    canvas.drawRightString(19 * cm, 28 * cm, f"Exercice : {doc.exercice if hasattr(doc, 'exercice') else ''}")
    canvas.restoreState()

def pied_de_page(canvas, doc):
    canvas.saveState()
    canvas.setFont('Helvetica', 8)
    canvas.drawString(2 * cm, 1.5 * cm, "Conseil syndical Monica")
    num_page = canvas.getPageNumber()
    canvas.drawRightString(19 * cm, 1.5 * cm, f"Page {num_page}")
    canvas.restoreState()

# Pour suivre la pagination :
def on_page(canvas, doc):
    entete(canvas, doc)
    pied_de_page(canvas, doc)
    
# Exemple de génération
def creer_pdf():
    doc = SimpleDocTemplate("./resultats/exemple_1.pdf", pagesize=A4)
    styles = getSampleStyleSheet()
    contenu = []

    for i in range(60):
        contenu.append(Paragraph(f"Paragraphe {i+1}", styles['Normal']))
        contenu.append(Spacer(1,5))

    doc.build(contenu, onFirstPage=on_page, onLaterPages=on_page)

creer_pdf()


# Exemple niveau 2  
Intégration du nombre pages total dans la numérotation des pages

In [ ]:
import io
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from PyPDF2 import PdfReader  # pip install PyPDF2

def creer_pdf():
    styles = getSampleStyleSheet()
    contenu = []
    def creer_contenu():
        for i in range(80):
            contenu.append(Paragraph(f"Paragraphe {i+1}", styles['Normal']))
            contenu.append(Spacer(1, 5))

    total_pages = {'n': 0}

    def entete(canvas, doc):
        canvas.saveState()
        canvas.setFont('Helvetica-Bold', 10)
        canvas.drawString(2 * cm, 28 * cm, "Copropriété Monica – Historique RGD")
        canvas.restoreState()

    def pied_de_page(canvas, doc):
        canvas.saveState()
        canvas.setFont('Helvetica', 8)
        page_num = canvas.getPageNumber()
        total = total_pages['n']
        texte = f"Page {page_num}" if not total else f"Page {page_num} / {total}"
        canvas.drawString(2 * cm, 1.5 * cm, "Conseil syndical Monica")
        canvas.drawRightString(19 * cm, 1.5 * cm, texte)
        canvas.restoreState()

    def on_page(canvas, doc):
        entete(canvas, doc)
        pied_de_page(canvas, doc)

    # --- Première passe : écrire un PDF en mémoire (avec callbacks pour mise en page) ---
    creer_contenu()
    buffer_temp = io.BytesIO()
    temp_doc = SimpleDocTemplate(buffer_temp, pagesize=A4)
    temp_doc.build(contenu, onFirstPage=on_page, onLaterPages=on_page)

    # Calcul du nombre de pages
    # Revenir au début du buffer avant lecture
    buffer_temp.seek(0)
    # --- Lire le PDF en mémoire pour compter les pages (PyPDF2) ---
    reader = PdfReader(buffer_temp)
    page_count = len(reader.pages)
    total_pages['n'] = page_count

    # --- Seconde passe : reconstruction finale (affiche Page i / X) ---
    # Après le premier build, la variable 'contenu' a été vidée de ses données, il faut la régénérer.
    creer_contenu()
    buffer_final = io.BytesIO()
    final_doc = SimpleDocTemplate(buffer_final, pagesize=A4)
    final_doc.build(contenu, onFirstPage=on_page, onLaterPages=on_page)

    # Sauvegarde sur disque
    with open("exemple_2.pdf", "wb") as f:
        f.write(buffer_final.getvalue())

    buffer_temp.close()
    buffer_final.close()

creer_pdf()
